In [17]:
import sys
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os
from os import path
from pathlib import Path
import os
import json
import rasterio
import urllib.request
import pystac
import numpy as np
from osgeo import gdal
from datetime import datetime, timezone
from shapely.geometry import Polygon, mapping
from tempfile import TemporaryDirectory

In [52]:
#set up logging 
logging.basicConfig(level=logging.info)

debug=logging.debug
info=logging.info
warning=logging.warning
error=logging.error

In [41]:
#list .laz objects in bucket
object_key=r"STAC_LiDAR/DEM/"

#s3 storage location for json files
json_out_loc=r'STAC_LiDAR/JSON/'

In [42]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)
# for some reason the bucket is adding an extra letter at the end???
bucket = constants.AWS_S3_BUCKET

DEBUG:botocore.hooks:Event choose-service-name: calling handler <function handle_service_name_alias at 0x7f1ed944f920>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x7f1ed93ac400>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function lazy_call.<locals>._handler at 0x7f1e4e1a9620>
DEBUG:botocore.hooks:Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x7f1ed93ac180>
DEBUG:botocore.endpoint:Setting s3 timeout as (60, 60)
DEBUG:botocore.client:Registering retry handlers for service: s3
DEBUG:botocore.utils:Registering S3 region redirector handler
DEBUG:botocore.utils:Registering S3Express Identity Resolver


In [54]:
DEM_Objects=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=object_key, StartAfter=object_key)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        DEM_Objects.append(obj['Key'])
        info(obj['Key'])
        info(f"Object Size {obj['Size']}")
        
else:
    error("No objects found in the bucket.")

DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function set_list_objects_encoding_type_url at 0x7f1ed926e7a0>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function validate_bucket_name at 0x7f1ed926d300>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function remove_bucket_from_url_paths_from_model at 0x7f1ed926f420>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <bound method S3RegionRedirectorv2.annotate_request_context of <botocore.utils.S3RegionRedirectorv2 object at 0x7f1e4c4acd10>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <bound method S3ExpressIdentityResolver.inject_signing_cache_key of <botocore.utils.S3ExpressIdentityResolver object at 0x7f1e4794cf10>>
DEBUG:botocore.hooks:Event before-parameter-build.s3.ListObjectsV2: calling handler <function generate_idempotent_uuid at 0x7f1ed926d

In [70]:
#set gdal s3 config 
# gdal.SetConfigOption('AWS_REGION', '443')
gdal.SetConfigOption('AWS_S3_ENDPOINT', constants.AWS_S3_ENDPOINT)
gdal.SetConfigOption('AWS_SECRET_ACCESS_KEY', constants.AWS_SECRET_ACCESS_KEY)
gdal.SetConfigOption('AWS_ACCESS_KEY_ID', constants.AWS_ACCESS_KEY_ID)
gdal.SetConfigOption('AWS_S3_BUCKET',constants.AWS_S3_BUCKET)



In [71]:
path= f'/vsis3/{bucket}/STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o029_xli1m_utm10_2019_cog.tif'
info(path)
ds=gdal.Open(path)


INFO:root:/vsis3/rczimv/STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o029_xli1m_utm10_2019_cog.tif
Warning 1: HTTP response code on https://rczimv.https://nrs.objectstore.gov.bc.ca:443///STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o029_xli1m_utm10_2019_cog.tif: 0
ERROR 4: `/vsis3/rczimv/STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o029_xli1m_utm10_2019_cog.tif' does not exist in the file system, and is not recognized as a supported dataset name.


In [ ]:
tmp_dir = TemporaryDirectory()